# Full Hybrid Pipeline: Thai-English NLP System
*Typhoon Translate 4B (GGUF) + XLM-RoBERTa Hierarchical Classifier + Typhoon 2.1 Instruct (4B)*

## Overview
This notebook implements a complete hybrid NLP pipeline for Thai-to-English grammar analysis with three components:
1. **Translation**: Typhoon Translate 4B (GGUF) for Thai → English translation
2. **Classification**: Custom XLM-RoBERTa model for hierarchical tense classification
3. **Explanation**: Typhoon 2.1 Instruct (4B) for detailed grammar explanations

### Pipeline Flow
Thai Input → Translation → Tense Classification → Grammar Explanation → Structured Output

---
## Section 1: Environment Setup

In [ ]:
# 1.1 Install Build Tools (Optional - only if CUDA wheel not available)
!apt-get update -y && apt-get install -y build-essential cmake ninja-build
!pip install -U 'pip>=23.3'

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,758 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,103 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,420 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy

In [ ]:
# 1.2 Install llama-cpp-python with CUDA support
import os, re, subprocess, sys

# Detect CUDA version and attempt wheel installation
CUDA_VERSION = !nvidia-smi --query-gpu=cuda_version --format=csv,noheader
CUDA_VERSION = CUDA_VERSION[0].strip()
cu_tag = re.sub(r"\.", "", CUDA_VERSION)[:3]

extra_index = f"https://abetlen.github.io/llama-cpp-python/whl/cu{cu_tag}"
print(f"🔍 Trying prebuilt wheel from: {extra_index}")

try:
    subprocess.check_call([
        sys.executable, "-m", "pip", "install", "llama-cpp-python",
        "--extra-index-url", extra_index
    ])
    print("✅ Prebuilt wheel installed successfully")
except subprocess.CalledProcessError:
    print("⚠️ Prebuilt wheel not found - compiling from source...")
    gpu_info = !nvidia-smi
    arch = "80" if any("A100" in line for line in gpu_info) else "75"

    # Clean setup for compilation
    !rm -rf llama.cpp
    !git clone https://github.com/ggerganov/llama.cpp.git
    %cd llama.cpp

    # Configure build environment
    env = os.environ.copy()
    env["CMAKE_ARGS"] = f"-DLLAMA_CUDA=on -DLLAMA_CUBLAS=on -DLLAMA_GEMMA=on -DCMAKE_CUDA_ARCHITECTURES={arch}"
    env["FORCE_CMAKE"] = "1"

    subprocess.check_call([
        sys.executable, "-m", "pip", "install", "--no-cache-dir",
        "--force-reinstall", "-e", "."
    ], env=env)

# Install core dependencies
!pip install -q -U huggingface_hub transformers accelerate safetensors

print("✅ llama-cpp-python and dependencies ready")

🔍 Trying prebuilt wheel from: https://abetlen.github.io/llama-cpp-python/whl/cuFie
✅ Prebuilt wheel installed successfully
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 167.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 177.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 210.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 161.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 152.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 194.8 MB/s eta

In [ ]:
# 1.3 Mount Google Drive and Setup Directories
from google.colab import drive
import os, torch, gc, time

drive.mount("/content/drive")

# Define project structure
BASE_DIR = "/content/drive/MyDrive/Dissertation/Model v.5"
MODEL_DIR = f"{BASE_DIR}/models"
RESULTS_DIR = f"{BASE_DIR}/results"
CACHE_DIR = f"{BASE_DIR}/cache"

# Create directories
for directory in [MODEL_DIR, RESULTS_DIR, CACHE_DIR]:
    os.makedirs(directory, exist_ok=True)

# Optimize PyTorch settings
torch.set_float32_matmul_precision("high")

print("✅ Google Drive mounted and directories configured")
print(f"📁 Base directory: {BASE_DIR}")
print(f"📁 Models: {MODEL_DIR}")
print(f"📁 Results: {RESULTS_DIR}")

Mounted at /content/drive
✅ Google Drive mounted and directories configured
📁 Base directory: /content/drive/MyDrive/Dissertation/Model v.5
📁 Models: /content/drive/MyDrive/Dissertation/Model v.5/models
📁 Results: /content/drive/MyDrive/Dissertation/Model v.5/results


---
## Section 2: Model Architecture Definitions

In [ ]:
# 2.1 Hierarchical BERT Classifier Architecture
import torch.nn as nn
from transformers import XLMRobertaModel, AutoConfig, PreTrainedModel

class XLMRHierClassifier(PreTrainedModel):
    """
    Hierarchical XLM-RoBERTa classifier for tense classification
    Predicts both coarse (Past/Present/Future) and fine-grained tense labels
    """
    config_class = AutoConfig

    def __init__(self, config, n_coarse=3, n_fine=25, coarse_w=0.3):
        super().__init__(config)
        self.encoder = XLMRobertaModel(config, add_pooling_layer=False)
        h = self.encoder.config.hidden_size

        # Dual classification heads
        self.coarse_head = nn.Linear(h, n_coarse)  # Past/Present/Future
        self.fine_head = nn.Linear(h, n_fine)      # Detailed tense categories

        self.crit = nn.CrossEntropyLoss()
        self.coarse_w = coarse_w  # Loss weighting
        self.post_init()

    def forward(self, input_ids=None, attention_mask=None, labels=None, **_):
        # Encode input
        hidden = self.encoder(input_ids, attention_mask=attention_mask).last_hidden_state
        pooled = hidden[:, 0]  # Use CLS token

        # Predict both levels
        logits_c = self.coarse_head(pooled)
        logits_f = self.fine_head(pooled)

        if labels is None:
            return {"logits": (logits_c, logits_f)}

        # Calculate hierarchical loss
        lab_c, lab_f = labels[:, 0], labels[:, 1]
        loss_c = self.crit(logits_c, lab_c)

        mask = lab_f != -100
        if mask.any():
            loss_f = self.crit(logits_f[mask], lab_f[mask])
            loss = self.coarse_w * loss_c + (1 - self.coarse_w) * loss_f
        else:
            loss = loss_c

        return {"loss": loss, "logits": (logits_c, logits_f)}

print("✅ Hierarchical classifier architecture defined")

✅ Hierarchical classifier architecture defined


In [ ]:
# 2.2 Download and Load Models
from huggingface_hub import hf_hub_download
from transformers import AutoTokenizer, AutoConfig
from safetensors.torch import load_file as safe_load_file
import json

# Model paths and configurations
BERT_CLS_PATH = f"{BASE_DIR}/bert-tense-hier/best"
TRANS_REPO = "scb10x/typhoon-translate-4b-gguf"
TRANS_FILE = "typhoon-translate-4b-q4_k_m.gguf"
TRANS_PATH = f"{MODEL_DIR}/{TRANS_FILE}"
EXPL_REPO = "scb10x/typhoon2.1-gemma3-4b"
EXPL_DIR = f"{MODEL_DIR}/typhoon2.1-gemma3-4b"

print("📥 Downloading/verifying models...")

# Download GGUF translator if needed
if not os.path.exists(TRANS_PATH):
    print("⬇️ Downloading GGUF translator...")
    hf_hub_download(
        repo_id=TRANS_REPO,
        filename=TRANS_FILE,
        local_dir=MODEL_DIR,
        resume_download=True
    )
    print(f"✅ Translator saved to {TRANS_PATH}")
else:
    print(f"✅ GGUF translator already exists")

# Download explainer model if needed
if not os.path.exists(EXPL_DIR):
    print("⬇️ Downloading explainer model...")
    from transformers import AutoTokenizer, AutoModelForCausalLM
    tokenizer = AutoTokenizer.from_pretrained(EXPL_REPO, cache_dir=EXPL_DIR)
    model = AutoModelForCausalLM.from_pretrained(EXPL_REPO, cache_dir=EXPL_DIR)
    print(f"✅ Explainer downloaded to {EXPL_DIR}")
else:
    print(f"✅ Explainer model already exists")

# Load BERT classifier
print(f"📂 Loading BERT classifier from: {BERT_CLS_PATH}")
config = AutoConfig.from_pretrained(BERT_CLS_PATH)
config.add_pooling_layer = False

bert_cls = XLMRHierClassifier(
    config=config,
    n_coarse=3,
    n_fine=25,
    coarse_w=0.3
)

# Load weights
weights = safe_load_file(os.path.join(BERT_CLS_PATH, "model.safetensors"))
bert_cls.load_state_dict(weights)

# Move to device
device = "cuda" if torch.cuda.is_available() else "cpu"
bert_cls.eval().to(device)

# Load label mappings
with open(f"{BERT_CLS_PATH}/coarse_labels.json") as f:
    coarse_labels = json.load(f)
with open(f"{BERT_CLS_PATH}/fine_labels.json") as f:
    fine_labels = json.load(f)

print(f"✅ BERT classifier loaded on {device}")
print(f"✅ Label mappings loaded: {len(coarse_labels)} coarse, {len(fine_labels)} fine labels")

📥 Downloading/verifying models...
✅ GGUF translator already exists
✅ Explainer model already exists
📂 Loading BERT classifier from: /content/drive/MyDrive/Dissertation/Model v.5/bert-tense-hier/best
✅ BERT classifier loaded on cuda
✅ Label mappings loaded: 3 coarse, 24 fine labels


In [ ]:
# 2.3 Test BERT Classifier
import torch
import torch.nn.functional as F

# Load tokenizer for testing
bert_tokenizer = AutoTokenizer.from_pretrained(BERT_CLS_PATH)

# Test with sample sentence
test_sentence = "I used to go to school"
print(f"🧪 Testing classifier with: '{test_sentence}'")

# Tokenize and predict
inputs = bert_tokenizer(test_sentence, return_tensors="pt").to(device)

with torch.no_grad():
    logits = bert_cls(**inputs)["logits"]
    coarse_logits, fine_logits = logits

# Convert to probabilities
coarse_probs = F.softmax(coarse_logits, dim=1)
fine_probs = F.softmax(fine_logits, dim=1)

# Get top 3 predictions
coarse_topk = torch.topk(coarse_probs, k=3, dim=1)
fine_topk = torch.topk(fine_probs, k=3, dim=1)

print("\n🏷️ Top 3 Coarse Predictions:")
for i in range(3):
    idx = coarse_topk.indices[0][i].item()
    prob = coarse_topk.values[0][i].item()
    print(f"   {i+1}. {coarse_labels[idx]} ({prob:.2%})")

print("\n🏷️ Top 3 Fine-grained Predictions:")
for i in range(3):
    idx = fine_topk.indices[0][i].item()
    prob = fine_topk.values[0][i].item()
    print(f"   {i+1}. {fine_labels[idx]} ({prob:.2%})")

print("\n✅ BERT classifier working correctly")

🧪 Testing classifier with: 'I used to go to school'

🏷️ Top 3 Coarse Predictions:
   1. Past (99.70%)
   2. Future (0.16%)
   3. Present (0.14%)

🏷️ Top 3 Fine-grained Predictions:
   1. NORFIN (99.94%)
   2. DOINGATSOMETIMEPAST (0.01%)
   3. HABIT (0.01%)

✅ BERT classifier working correctly


---
## Section 3: Complete Hybrid Pipeline System

### 3.1 Tense Definitions and Classifications

In [ ]:
# 3.2 Complete Hybrid Pipeline Implementation
import torch, json, time, re
from safetensors.torch import load_file as safe_load_file
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
from llama_cpp import Llama

class TenseTagDefinitions:
    """Comprehensive tense tag definitions for grammar explanations"""

    def __init__(self):
        # Coarse-level definitions
        self.coarse_definitions = {
            "PAST": "หมายถึง เหตุการณ์หรือการกระทำที่เกิดขึ้นและจบลงแล้วในอดีต",
            "PRESENT": "หมายถึง เหตุการณ์หรือการกระทำที่เกิดขึ้นในปัจจุบัน หรือเป็นความจริงทั่วไป",
            "FUTURE": "หมายถึง เหตุการณ์หรือการกระทำที่จะเกิดขึ้นในอนาคต"
        }

        # Fine-grained tense definitions with detailed explanations
        self.fine_definitions = {
            # Present Simple categories
            "HABIT": {
                "tense": "Present Simple",
                "thai_name": "Present Simple - กิจวัตร/นิสัย",
                "usage": "ใช้เมื่อพูดถึงกิจวัตรหรือพฤติกรรมที่ทำเป็นประจำ",
                "structure": "Subject + V1 (ถ้าประธานเอกพจน์เติม s/es)",
                "keywords": "always, usually, often, sometimes, every day",
                "example": "I drink coffee every morning."
            },
            "FACT": {
                "tense": "Present Simple",
                "thai_name": "Present Simple - ข้อเท็จจริง",
                "usage": "ใช้กับข้อเท็จจริงที่เป็นสัจธรรมหรือเป็นความรู้ทางวิทยาศาสตร์",
                "structure": "Subject + V1",
                "keywords": "ข้อเท็จจริงทั่วไป, สัจธรรม",
                "example": "The sun rises in the east."
            },
            "SCHEDULEDFUTURE": {
                "tense": "Present Simple",
                "thai_name": "Present Simple - ตารางเวลา/แผนการที่กำหนดไว้",
                "usage": "ใช้เมื่อกล่าวถึงตารางเวลา ตารางเดินรถ แผนที่กำหนดไว้แน่นอน หรือแผนการในอนาคตที่วางไว้",
                "structure": "Subject + V1",
                "keywords": "schedule, timetable, plan to, intend to",
                "example": "The train leaves at 9 AM. / I plan to study abroad next year."
            },
            "SAYING": {
                "tense": "Present Simple",
                "thai_name": "Present Simple - สุภาษิต/คำพังเพย",
                "usage": "ใช้กับสุภาษิต คำพังเพย หรือคำกล่าวทั่วไป",
                "structure": "Subject + V1",
                "keywords": "สุภาษิต, คำกล่าว",
                "example": "Practice makes perfect."
            },
            "HEADLINE": {
                "tense": "Present Simple",
                "thai_name": "Present Simple - พาดหัวข่าว",
                "usage": "ใช้ในพาดหัวข่าวหรือข้อความที่เน้นย่อประโยค แม้จะพูดถึงเหตุการณ์ที่เกิดขึ้นแล้ว เพื่อสร้างความรู้สึกสดใหม่",
                "structure": "Subject + V1",
                "keywords": "พาดหัวข่าว",
                "example": "Prime Minister visits flood victims."
            },

            # Present Continuous categories
            "HAPPENING": {
                "tense": "Present Continuous",
                "thai_name": "Present Continuous - กำลังเกิดขึ้น",
                "usage": "สิ่งที่ทำอยู่ขณะพูด",
                "structure": "Subject + is/am/are + V-ing",
                "keywords": "now, right now, at the moment",
                "example": "I am writing an email now."
            },
            "NOWADAYS": {
                "tense": "Present Continuous",
                "thai_name": "Present Continuous - ช่วงนี้",
                "usage": "สิ่งที่ทำอยู่ในช่วงนี้ เช่น โปรเจค หรือสิ่งที่ใช้เวลาทำนานเป็นหลักวัน",
                "structure": "Subject + is/am/are + V-ing",
                "keywords": "these days, nowadays, currently",
                "example": "I am working on a big project these days."
            },
            "SUREFUT": {
                "tense": "Present Continuous",
                "thai_name": "Present Continuous - อนาคตที่วางแผนไว้",
                "usage": "เหตุการณ์ที่จะเกิดขึ้นในอนาคตโดยมีการวางแผนไว้แล้ว มักเจอ be + going to",
                "structure": "Subject + is/am/are + going to + V1",
                "keywords": "tomorrow, next week, planning",
                "example": "I am going to visit my parents tomorrow."
            },
            "PROGRESS": {
                "tense": "Present Continuous",
                "thai_name": "Present Continuous - กำลังเปลี่ยนแปลง",
                "usage": "เหตุการณ์ที่กำลังมีการเปลี่ยนแปลง พัฒนาขึ้น หรือก้าวหน้าขึ้น",
                "structure": "Subject + is/am/are + V-ing",
                "keywords": "changing, improving, getting better",
                "example": "The weather is getting warmer."
            },

            # Present Perfect categories
            "JUSTFIN": {
                "tense": "Present Perfect",
                "thai_name": "Present Perfect - เพิ่งจบ",
                "usage": "ใช้เมื่อเหตุการณ์เพิ่งจะสิ้นสุดลง",
                "structure": "Subject + have/has + V3",
                "keywords": "just, just now",
                "example": "I have just finished my homework."
            },
            "RESULT": {
                "tense": "Present Perfect",
                "thai_name": "Present Perfect - มีผลถึงปัจจุบัน",
                "usage": "สิ่งที่เกิดขึ้นตั้งแต่อดีตและมีผลหรือคงสภาพจนถึงปัจจุบัน",
                "structure": "Subject + have/has + V3",
                "keywords": "already, yet, still",
                "example": "I have lost my keys."
            },
            "EXP": {
                "tense": "Present Perfect",
                "thai_name": "Present Perfect - ประสบการณ์",
                "usage": "ประสบการณ์ (เจอคำว่า First / ... time)",
                "structure": "Subject + have/has + V3",
                "keywords": "ever, never, first time",
                "example": "This is the first time I have visited Japan."
            },

            # Present Perfect Continuous
            "SINCEFOR": {
                "tense": "Present Perfect Continuous",
                "thai_name": "Present Perfect Continuous - ทำมาตั้งแต่",
                "usage": "สิ่งที่ทำเรื่อยมาจนถึงปัจจุบัน โดยเน้นระยะเวลา มักเจอ for/since",
                "structure": "Subject + have/has + been + V-ing",
                "keywords": "for, since, all day",
                "example": "I have been studying for 3 hours."
            },

            # Past Simple
            "NORFIN": {
                "tense": "Past Simple",
                "thai_name": "Past Simple - อดีตทั่วไป",
                "usage": "การกระทำในอดีตทั่วไป โดยไม่มีบริบทหรือรายละเอียดเพิ่มเติม",
                "structure": "Subject + V2",
                "keywords": "yesterday, last week, ago",
                "example": "I went to school yesterday."
            },

            # Past Continuous
            "INTERRUPT": {
                "tense": "Past Continuous",
                "thai_name": "Past Continuous - ถูกขัดจังหวะ",
                "usage": "ใช้คู่กับ Past Simple เพื่อบอกว่าเหตุการณ์ใน Past simple เกิดแทรกอีกเหตุการณ์ที่กำลังทำในอดีต",
                "structure": "Subject + was/were + V-ing + when + Past Simple",
                "keywords": "when, while",
                "example": "I was sleeping when the phone rang."
            },
            "DOINGATSOMETIMEPAST": {
                "tense": "Past Continuous",
                "thai_name": "Past Continuous - กำลังทำในอดีต",
                "usage": "สิ่งที่กำลังทำอยู่ ณ เวลาหนึ่งในอดีต",
                "structure": "Subject + was/were + V-ing",
                "keywords": "at ... yesterday, at that time",
                "example": "I was reading at 8 PM yesterday."
            },

            # Past Perfect
            "BEFOREPAST": {
                "tense": "Past Perfect",
                "thai_name": "Past Perfect - ก่อนเหตุการณ์ในอดีต",
                "usage": "ใช้เมื่อต้องการแสดงว่าเหตุการณ์หนึ่งเกิดขึ้นและเสร็จก่อนอีกเหตุการณ์ในอดีต",
                "structure": "Subject + had + V3",
                "keywords": "before, after, already",
                "example": "She had finished homework before she ate dinner."
            },

            # Past Perfect Continuous
            "DURATION": {
                "tense": "Past Perfect Continuous",
                "thai_name": "Past Perfect Continuous - ทำมาก่อนในอดีต",
                "usage": "ใช้คู่กับ Past Simple เพื่อบอกสิ่งที่กำลังทำอยู่สักพักหนึ่งก่อนอีกสิ่งจะเกิดในอดีต",
                "structure": "Subject + had + been + V-ing",
                "keywords": "for, since, before",
                "example": "I had been waiting for 2 hours before he arrived."
            },

            # Future Simple
            "50PERC": {
                "tense": "Future Simple",
                "thai_name": "Future Simple - คาดการณ์ 50%",
                "usage": "สิ่งที่คาดจะเกิด หรือมีแนวโน้มในอนาคต (50%)",
                "structure": "Subject + will + V1",
                "keywords": "probably, maybe, I think",
                "example": "It will probably rain tomorrow."
            },
            "PROMISE": {
                "tense": "Future Simple",
                "thai_name": "Future Simple - สัญญา/เสนอ",
                "usage": "การให้คำสัญญา หรือเสนออะไรให้ใคร",
                "structure": "Subject + will + V1",
                "keywords": "promise, offer",
                "example": "I will help you with your homework."
            },
            "RIGHTNOW": {
                "tense": "Future Simple",
                "thai_name": "Future Simple - ตัดสินใจทันที",
                "usage": "สิ่งที่เพิ่งคิดว่าจะทำเดี๋ยวนั้น (ไม่ได้วางแผนว่าจะทำมาก่อน)",
                "structure": "Subject + will + V1",
                "keywords": "OK, I'll..., spontaneous decision",
                "example": "The doorbell is ringing. I'll answer it."
            },

            # Future Continuous
            "LONGFUTURE": {
                "tense": "Future Continuous",
                "thai_name": "Future Continuous - กำลังทำในอนาคต",
                "usage": "สิ่งที่คาดว่าน่าจะเกิดขึ้น คงจะทำอยู่ หรือวางแผนว่าจะทำ ณ เวลาหนึ่งในอนาคต",
                "structure": "Subject + will + be + V-ing",
                "keywords": "at ... tomorrow, this time next week",
                "example": "I will be studying at 8 PM tomorrow."
            },

            # Future Perfect
            "PREDICT": {
                "tense": "Future Perfect",
                "thai_name": "Future Perfect - จะเสร็จก่อน",
                "usage": "สิ่งที่คาดว่าคงจบ เสร็จสิ้น หรือครบเวลาแล้ว ณ เวลาหนึ่งในอนาคต",
                "structure": "Subject + will + have + V3",
                "keywords": "by, by the time",
                "example": "I will have finished by 5 PM."
            },

            # Future Perfect Continuous
            "WILLCONTINUEINFUTURE": {
                "tense": "Future Perfect Continuous",
                "thai_name": "Future Perfect Continuous - ทำต่อเนื่องในอนาคต",
                "usage": "สิ่งที่คาดว่าคงจบ เสร็จสิ้น หรือครบเวลาแล้ว ณ เวลาหนึ่งในอนาคต และจะคงสภาพหรือทำแบบนี้ต่อไปอีก",
                "structure": "Subject + will + have + been + V-ing",
                "keywords": "for, by the time",
                "example": "By next year, I will have been working here for 10 years."
            }
        }

class Hybrid4BSystem:
    """
    Complete hybrid NLP pipeline system combining:
    - GGUF Translator (Typhoon Translate 4B)
    - BERT Classifier (XLM-RoBERTa Hierarchical)
    - Instruct Explainer (Typhoon 2.1 4B)
    """

    def __init__(self):
        # Model paths
        self.gguf_path = TRANS_PATH
        self.exp_dir = EXPL_DIR
        self.cls_dir = BERT_CLS_PATH

        # Model instances
        self.translator = None
        self.expl_tokenizer = None
        self.expl_model = None
        self.cls_tokenizer = None
        self.cls_model = None

        # Device configuration
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

        # Initialize components
        self.tense_defs = TenseTagDefinitions()
        self._load_label_mappings()
        self._setup_examples()

        # Caching for efficiency
        self.translation_cache = {}
        self.explanation_cache = {}
        self.max_cache_size = 100

    def _load_label_mappings(self):
        """Load tense label mappings"""
        with open(f"{self.cls_dir}/fine_labels.json") as f:
            self.id2fine = json.load(f)
        with open(f"{self.cls_dir}/coarse_labels.json") as f:
            self.id2coarse = json.load(f)

    def _setup_examples(self):
        """Setup few-shot examples for improved translation"""
        self.translation_examples = [
            {
                "thai": "ฉันกินข้าวเป็นประจำทุกเช้า",
                "english": "I eat breakfast every morning.",
                "fine": "HABIT",
                "coarse": "PRESENT"
            },
            {
                "thai": "เขากำลังทำงานอยู่ตอนนี้",
                "english": "He is working right now.",
                "fine": "HAPPENING",
                "coarse": "PRESENT"
            },
            {
                "thai": "ฉันเพิ่งทำการบ้านเสร็จ",
                "english": "I have just finished my homework.",
                "fine": "JUSTFIN",
                "coarse": "PRESENT"
            }
        ]

    def load_translator(self, n_gpu_layers=-1):
        """Load GGUF translator model"""
        if self.translator is None:
            print("🔌 Loading GGUF translator...")
            self.translator = Llama(
                model_path=self.gguf_path,
                n_gpu_layers=n_gpu_layers,
                n_ctx=4096,
                n_batch=512,
                f16_kv=True,
                use_mlock=True,
                use_mmap=True,
                logits_all=False,
                verbose=False
            )
            print("✅ Translator loaded successfully")

    def load_explainer(self):
        """Load Transformers explainer model"""
        if self.expl_model is None:
            print("🔌 Loading explainer model...")
            # Configure torch compilation
            import torch._dynamo
            torch._dynamo.config.suppress_errors = True
            torch._dynamo.config.cache_size_limit = 64

            self.expl_tokenizer = AutoTokenizer.from_pretrained(self.exp_dir, use_fast=True)
            self.expl_model = AutoModelForCausalLM.from_pretrained(
                self.exp_dir,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                low_cpu_mem_usage=True,
                attn_implementation="eager"
            ).eval()
            print("✅ Explainer loaded successfully")

    def load_classifier(self):
        """Load BERT classifier model"""
        if self.cls_model is None:
            print("🔌 Loading BERT classifier...")
            self.cls_tokenizer = AutoTokenizer.from_pretrained(self.cls_dir)
            config = AutoConfig.from_pretrained(self.cls_dir)
            self.cls_model = XLMRHierClassifier(config, n_coarse=3, n_fine=25, coarse_w=0.3)
            weights = safe_load_file(f"{self.cls_dir}/model.safetensors")
            self.cls_model.load_state_dict(weights)
            self.cls_model.eval().to(self.device)
            print("✅ Classifier loaded successfully")

    def translate(self, thai_text, max_tokens=80):
        """Translate Thai text to English"""
        if self.translator is None:
            raise RuntimeError("Translator not loaded")

        # Check cache
        if thai_text in self.translation_cache:
            return self.translation_cache[thai_text]

        # Simple prompt for non-instruct model
        prompt = f"""Translate the following Thai sentence to English:
Thai: {thai_text}
English:"""

        start_time = time.time()
        output = self.translator(
            prompt,
            max_tokens=max_tokens,
            temperature=0.1,
            top_p=0.95,
            top_k=40,
            repeat_penalty=1.1,
            stop=["\n", "Thai:", "Translate"]
        )
        duration = time.time() - start_time

        # Clean output
        translation = output["choices"][0]["text"].strip()
        translation = translation.split("\n")[0]

        # Cache result
        if len(self.translation_cache) >= self.max_cache_size:
            self.translation_cache.pop(next(iter(self.translation_cache)))
        self.translation_cache[thai_text] = (translation, duration)

        return translation, duration

    def predict_tense(self, english_sentence, top_k=3):
        """Predict tense labels with confidence scores"""
        if self.cls_model is None or self.cls_tokenizer is None:
            raise RuntimeError("Classifier not loaded")

        inputs = self.cls_tokenizer(english_sentence, return_tensors="pt").to(self.device)

        with torch.inference_mode():
            logits = self.cls_model(**inputs)["logits"]
            coarse_logits, fine_logits = logits

        # Convert to probabilities
        coarse_probs = torch.nn.functional.softmax(coarse_logits, dim=1)
        fine_probs = torch.nn.functional.softmax(fine_logits, dim=1)

        # Get top-k predictions
        coarse_topk = torch.topk(coarse_probs, k=top_k, dim=1)
        fine_topk = torch.topk(fine_probs, k=top_k, dim=1)

        # Format results
        coarse_result = [(self.id2coarse[i.item()], p.item())
                        for i, p in zip(coarse_topk.indices[0], coarse_topk.values[0])]
        fine_result = [(self.id2fine[i.item()], p.item())
                      for i, p in zip(fine_topk.indices[0], fine_topk.values[0])]

        return coarse_result, fine_result

    def analyze_sentence_context(self, thai_text, english_translation):
        """Analyze sentence context for better explanations"""
        # Determine sentence type
        if any(word in thai_text for word in ['อุณหภูมิ', 'องศา', 'โลก', 'ดวงอาทิตย์', 'วิทยาศาสตร์']):
            sentence_type = "ข้อเท็จจริงทางวิทยาศาสตร์"
        elif any(word in thai_text for word in ['ทุกวัน', 'เสมอ', 'ประจำ', 'ทุกเช้า']):
            sentence_type = "กิจวัตรประจำวัน"
        elif any(word in thai_text for word in ['มักจะ', 'ชอบ', 'เคย', 'บ่อย']):
            sentence_type = "นิสัยส่วนบุคคล"
        elif any(word in thai_text for word in ['กำลัง', 'อยู่', 'ตอนนี้']):
            sentence_type = "การกระทำที่กำลังเกิดขึ้น"
        else:
            sentence_type = "ประโยคทั่วไป"

        # Extract signal words
        thai_signals = {
            'ทุกวัน': 'every day', 'เสมอ': 'always', 'ปกติ': 'usually',
            'บ่อยๆ': 'often', 'ตอนนี้': 'now', 'กำลัง': 'currently',
            'เมื่อวาน': 'yesterday', 'พรุ่งนี้': 'tomorrow'
        }

        found_signals = []
        for thai, eng in thai_signals.items():
            if thai in thai_text:
                found_signals.append(f"{thai} ({eng})")

        return {
            "sentence_type": sentence_type,
            "signal_words": ", ".join(found_signals) if found_signals else "ไม่มีคำสัญญาณชัดเจน"
        }

    def explain(self, thai_text, english_translation, max_tokens=600):
        """Generate detailed grammar explanation"""
        if self.expl_model is None:
            raise RuntimeError("Explainer not loaded")

        # Get tense predictions
        coarse_preds, fine_preds = self.predict_tense(english_translation)

        print("🔍 [Tense Prediction]")
        print(f"   Coarse: {coarse_preds[0][0]} ({coarse_preds[0][1]:.2%})")
        print(f"   Fine  : {fine_preds[0][0]} ({fine_preds[0][1]:.2%})")

        fine_label = fine_preds[0][0]
        confidence = fine_preds[0][1]
        fine_def = self.tense_defs.fine_definitions.get(fine_label, {})

        # Build context-aware prompt
        tag_context = f"""
Tense ที่ตรวจพบ: {fine_label}
ประเภท: {fine_def.get('tense', 'Unknown')} - {fine_def.get('thai_name', '')}
การใช้งาน: {fine_def.get('usage', '')}
โครงสร้าง: {fine_def.get('structure', '')}
คำสัญญาณ: {fine_def.get('keywords', '')}
ตัวอย่าง: {fine_def.get('example', '')}
"""

        prompt_body = f"""<context>
คุณคือระบบวิเคราะห์ไวยากรณ์ภาษาอังกฤษสำหรับผู้เรียนไทย
คุณมีความรู้ลึกซึ้งเกี่ยวกับระบบ tense ในภาษาอังกฤษและความแตกต่างกับภาษาไทย
</context>

<tense_knowledge>
{tag_context}
</tense_knowledge>

<task>
วิเคราะห์การแปลประโยคและอธิบายการใช้ tense ที่เลือกอย่างละเอียด
</task>

<input>
ประโยคภาษาไทย: {thai_text}
การแปลภาษาอังกฤษ: {english_translation}
Tense ที่ระบบตรวจพบ: {fine_label} (ความมั่นใจ: {confidence:.1%})
</input>

<requirements>
โปรดอธิบายโดยครอบคลุมประเด็นต่อไปนี้:

1. **วิเคราะห์ Tense ที่ใช้**:
   - อธิบาย tense ทางไวยากรณ์ที่ใช้ (เช่น Present Simple, Past Perfect)
   - อธิบายการใช้งานในบริบทนี้โดยเฉพาะ
   - โครงสร้างไวยากรณ์: {fine_def.get('structure', '')}

2. **คำศัพท์ที่น่าสนใจ**:
   - เลือกคำศัพท์ / วลี ภาษาอังกฤษที่น่าสนใจจากประโยคมาหนึ่งคำ / วลี และอธิบายว่าทำไมถึงเลือกใช้คำนั้นในการแปล

3. **ข้อผิดพลาดที่พบบ่อย**:
   - ผู้เรียนไทยมักใช้ tense ที่ใช้ในประโยคผิดอย่างไร
   - วิธีจำง่าย ๆ

</requirements>

<format>
- ใช้ภาษาไทยที่เข้าใจง่าย
- อธิบายเป็นขั้นตอน มีหัวข้อชัดเจน
- ยกตัวอย่างประกอบ
- เน้นสิ่งที่ผู้เรียนไทยควรระวัง
- **เริ่มต้นด้วยการวิเคราะห์ทันที**
- **ใช้รูปแบบวิชาการ ไม่ใช่รูปแบบสนทนา**
- ห้ามเขียนหัวข้ออื่น ๆ ที่ไม่อยู่ใน Requirement
</format>"""

        messages = [
            {
                "role": "system",
                "content": """คุณคือระบบวิเคราะห์ไวยากรณ์ภาษาอังกฤษสำหรับผู้เรียนไทย คุณให้คำอธิบายที่ตรงประเด็น กระชับ และเป็นวิชาการ

กฎสำคัญ:
- อธิบาย TENSE ทางไวยากรณ์ (Present Simple, Past Perfect ฯลฯ) ไม่ใช่รหัสจัดหมวดหมู่
- ไม่ใช้คำทักทาย คำลา หรือบทนำ
- เริ่มต้นด้วยการวิเคราะห์ทันที
- ใช้ภาษาวิชาการที่เข้าใจง่าย
- ตอบตรงประเด็นตามหัวข้อที่กำหนด"""
            },
            {"role": "user", "content": prompt_body}
        ]

        # Generate explanation
        full_prompt = self.expl_tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )

        toks = self.expl_tokenizer(full_prompt, return_tensors="pt").to(self.expl_model.device)

        start_time = time.time()
        with torch.inference_mode():
            with torch.backends.cuda.sdp_kernel(enable_flash=False, enable_math=True, enable_mem_efficient=True):
                output = self.expl_model.generate(
                    **toks,
                    max_new_tokens=max_tokens,
                    do_sample=True,
                    temperature=0.7,
                    top_p=0.9,
                    top_k=50,
                    repetition_penalty=1.1,
                    pad_token_id=self.expl_tokenizer.eos_token_id,
                    eos_token_id=self.expl_tokenizer.eos_token_id,
                    use_cache=True
                )
        duration = time.time() - start_time

        explanation = self.expl_tokenizer.decode(
            output[0][toks['input_ids'].shape[-1]:],
            skip_special_tokens=True
        ).strip()

        # Validate explanation quality
        if self.validate_explanation(explanation, fine_label):
            print("✅ Explanation validation passed")
        else:
            print("⚠️ Explanation may be incomplete")

        return explanation, duration

    def validate_explanation(self, explanation, fine_label):
        """Validate explanation contains required elements"""
        required_elements = [
            ("tense_mention", [self.tense_defs.fine_definitions.get(fine_label, {}).get('tense', '').lower()]),
            ("structure", ["โครงสร้าง", "รูปประโยค", "subject", "verb"]),
            ("mistakes", ["ผิดพลาด", "ข้อผิด", "พลาด", "ระวัง"])
        ]

        missing = []
        for category, keywords in required_elements:
            if not any(keyword in explanation.lower() for keyword in keywords):
                missing.append(category)

        if missing:
            print(f"⚠️ Missing elements: {', '.join(missing)}")

        return len(missing) == 0

    def get_tense_info(self, fine_label):
        """Get detailed information about a tense tag"""
        if fine_label not in self.tense_defs.fine_definitions:
            return f"Unknown tense tag: {fine_label}"

        info = self.tense_defs.fine_definitions[fine_label]
        return f"""
Tense Tag: {fine_label}
Tense: {info.get('tense', 'Unknown')}
Thai Name: {info.get('thai_name', '')}
Usage: {info.get('usage', '')}
Structure: {info.get('structure', '')}
Keywords: {info.get('keywords', '')}
Example: {info.get('example', '')}
"""

    def full_pipeline(self, thai_text, verbose=True):
        """Execute complete pipeline: Translation → Classification → Explanation"""
        if verbose:
            print("="*60)
            print(f"📝 Input Thai: {thai_text}")
            print("="*60)

        # Step 1: Translation
        if verbose:
            print("\n🔄 Step 1: Translation")
        translation, trans_time = self.translate(thai_text)
        if not translation:
            raise ValueError("Translation failed")
        if verbose:
            print(f"   Result: {translation}")
            print(f"   Time: {trans_time:.2f}s")

        # Step 2: Tense Classification
        if verbose:
            print("\n🏷️ Step 2: Tense Classification")
        coarse_preds, fine_preds = self.predict_tense(translation)
        if verbose:
            print(f"   Top predictions:")
            for i, (label, conf) in enumerate(fine_preds[:3]):
                tense_info = self.tense_defs.fine_definitions.get(label, {})
                print(f"   {i+1}. {label} ({tense_info.get('thai_name', 'Unknown')}): {conf:.2%}")

        # Step 3: Grammar Explanation
        if verbose:
            print("\n💡 Step 3: Grammar Explanation")
        explanation, expl_time = self.explain(thai_text, translation)

        # Compile results
        result = {
            "thai_input": thai_text,
            "english_translation": translation,
            "tense_predictions": {
                "coarse": coarse_preds[0],
                "fine": fine_preds[0],
                "all_predictions": fine_preds
            },
            "tense_details": self.get_tense_info(fine_preds[0][0]),
            "explanation": explanation,
            "timing": {
                "translation": trans_time,
                "explanation": expl_time,
                "total": trans_time + expl_time
            }
        }

        if verbose:
            print(f"\n📊 Total processing time: {result['timing']['total']:.2f}s")
            print("\n" + "="*60)
            print("📖 TENSE DETAILS:")
            print(result['tense_details'])
            print("="*60)
            print("📖 EXPLANATION:")
            print("="*60)
            print(explanation)
            print("="*60)

        return result

print("✅ Complete Hybrid4BSystem implemented")

✅ Complete Hybrid4BSystem implemented


In [ ]:
if __name__ == "__main__":
    import pandas as pd
    import random
    from datetime import datetime

    # Initialize system
    system = Hybrid4BSystem()

    # Load all models
    system.load_translator()
    system.load_classifier()
    system.load_explainer()

    # Extended test sentences
    test_sentences = [
        "ฉันกินข้าวเช้าทุกวัน",                            # HABIT
        "เราไปวิ่งตอนเช้าเป็นประจำ",                     #  HABIT
        "เขาตื่นสายเสมอ",                             # HABIT
        "คุณพ่อของผมตื่นนอนเวลา 8 โมงเช้า", # HABIT
        "น้ำเปล่าเดือดที่อุณหภูมิ 100 องศา",     # FACT
        "โลกหมุนรอบดวงอาทิตย์",     # FACT
        "ดวงอาทิตย์ขึ้นทิศตะวันออก",     # FACT
        "หนึ่งปีมีทั้งหมด 365 วัน",   # FACT
        "รถไฟออกตอน 9 โมง",         # SCHEDULEDFUTURE
        "ฉันวางแผนจะเรียนต่อต่างประเทศปีหน้า",  # SCHEDULEDFUTURE
        "คุณวางแผนอนาคตว่าอย่างไรบ้าง", # SCHEDULEDFUTURE
        "เขาตั้งใจจะลาออกจากงาน",  # SCHEDULEDFUTURE
        "นายกเข้าให้กำลังใจประชาชนที่ประสบน้ำท่วม", # HEADLINE
        "ประเทศไทย ประสบปัญหาขาดความคล่องตัวทางการเงิน", # HEADLINE
        "สุนัขเป็นฮีโร่จากการเข้าช่วยเจ้าของ", # HEADLINE
        "นักวิทยาศาสตร์ค้นพบแร่หายากใหม่", # HEADLINE
        "น้ำขึ้นให้รีบตัก", # SAYING
        "ความพยายามอยู่ที่ไหน ความสำเร็จอยู่ที่นั่น", # SAYING
        "ช้าๆ ได้พร้าเล่มงาม", # SAYING
        "ทุกปัญหามีทางออก", # SAYING
        "เขากำลังอ่านหนังสืออยู่ ณ ตอนนี้",   # HAPPENING
        "ฝนกำลังตกตอนนี้",          # HAPPENING
        "คุณแม่กำลังล้างจานอยู่", # HAPPENING
        "แฟนสาวของฉันกำลังซักผ้า", # HAPPENING
        "ฉันกำลังทำโปรเจคเกี่ยวกับการศึกษาอยู่ช่วงนี้", #NOWADAYS
        "ช่วงนี้พี่สาวฉันออกกำลังกาย", #NOWADAYS
        "ฉันกำลังต่อเติมบ้านอยู่", #NOWADAYS
        "บริษัทฉันกำลังปรับฐานเงินเดือนให้ทุกคน", #NOWADAYS
        "ฉันกำลังจะไปสนามบิน", # SUREFUT
        "เขากำลังจะสอบพรุ่งนี้",  # SUREFUT
        "เรากำลังจะเริ่มประชุมใน 5 นาที",  # SUREFUT
        "พวกเขากำลังจะย้ายบ้านสัปดาห์หน้า",  # SUREFUT
        "เศรษฐกิจกำลังดีขึ้นเรื่อยๆ", #PROGRESS
        "ฉันกำลังปรับปรุงทักษะการเขียนของฉัน", #PROGRESS
        "ทีมกำลังพัฒนาเทคโนโลยีใหม่", #PROGRESS
        "สภาพอากาศกำลังอุ่นขึ้น", #PROGRESS
        "ฉันเพิ่งทำการบ้านเสร็จ",  # JUSTFIN
        "เธอเพิ่งกลับมาถึงบ้าน",  # JUSTFIN
        "เราเพิ่งกินข้าวเสร็จ",  # JUSTFIN
        "เขาเพิ่งออกจากห้องประชุม",  # JUSTFIN
        "ฉันทำกุญแจหาย",  # RESULT
        "เขาซื้อของออนไลน์ไปแล้ว",  # RESULT
        "เราดูหนังเรื่องนี้แล้ว",  # RESULT
        "เธอลืมกระเป๋าไว้ที่บ้าน",  # RESULT
        "ฉันเคยไปญี่ปุ่นมาแล้ว",  # EXP
        "นี่เป็นครั้งแรกที่ฉันได้เจอหิมะ",  # EXP
        "คุณเคยเห็นปรากฏการณ์นี้ไหม",  # EXP
        "เขาไม่เคยกินทุเรียน",  # EXP
        "ฉันเรียนภาษาอังกฤษมา 3 ปีแล้ว",  # SINCEFOR
        "เขาทำงานที่นี่มาตั้งแต่ปี 2015",  # SINCEFOR
        "เรารอรถเมล์มาตั้งแต่บ่ายโมง",  # SINCEFOR
        "ฉันออกกำลังกายมาเป็นเวลา 2 ชั่วโมง",  # SINCEFOR
        "เมื่อวานฉันไปโรงเรียน",  # NORFIN
        "เขากินข้าวเที่ยงตอนบ่ายสอง",  # NORFIN
        "เราไปดูหนังเมื่อคืน",  # NORFIN
        "ฉันซื้อน้ำมาเมื่อเช้า",  # NORFIN
        "ฉันกำลังนอนตอนที่เธอโทรมา",  # INTERRUPT
        "เขากำลังอาบน้ำเมื่อไฟดับ",  # INTERRUPT
        "เรากำลังดูหนังตอนที่เพื่อนมาถึง",  # INTERRUPT
        "แม่กำลังทำกับข้าวตอนที่พ่อกลับบ้าน",  # INTERRUPT
        "เมื่อคืนตอนสองทุ่มฉันกำลังดูหนัง",  # DOINGATSOMETIMEPAST
        "ตอนนั้นเขากำลังวิ่งอยู่",  # DOINGATSOMETIMEPAST
        "เวลา 5 โมงเมื่อวาน ฉันกำลังทำกับข้าว",  # DOINGATSOMETIMEPAST
        "ตอนนั้นเรากำลังประชุมกันอยู่",  # DOINGATSOMETIMEPAST
        "ฉันกินข้าวก่อนออกจากบ้าน",  # BEFOREPAST
        "เธออ่านหนังสือก่อนนอน",  # BEFOREPAST
        "เราทำการบ้านเสร็จก่อนดูทีวี",  # BEFOREPAST
        "เขาล้างจานเสร็จก่อนที่แม่จะกลับมา",  # BEFOREPAST
        "ฉันรอเขาอยู่เป็นชั่วโมงก่อนที่เขาจะมาถึง",  # DURATION
        "เขาทำงานที่บริษัทนี้มา 10 ปีก่อนจะลาออก",  # DURATION
        "เรารอฝนหยุดตกอยู่ครึ่งชั่วโมง ถึงจะได้กลับบ้าน",  # DURATION
        "เธอทำกับข้าวอยู่นานก่อนที่แขกจะมา",  # DURATION
        "ฉันจะไปเที่ยวพรุ่งนี้",  # 50PERC
        "พรุ่งนี้ฉันจะไปหาหมอ",  # 50PERC
        "ฝนคงจะตกตอนเย็น",  # 50PERC
        "เขาอาจจะไม่มางานเลี้ยง",  # 50PERC
        "ฉันจะช่วยคุณทำการบ้าน",  # PROMISE
        "ผมสัญญาว่าจะไม่ลืม",  # PROMISE
        "ฉันจะซื้อของขวัญให้เธอ",  # PROMISE
        "ผมจะโทรหาคุณคืนนี้",  # PROMISE
        "มีคนกดกริ่ง ฉันจะไปเปิดประตู",  # RIGHTNOW
        "ฉันเหนื่อยแล้ว ฉันจะนอนละ",  # RIGHTNOW
        "ตกลง ฉันจะช่วยคุณ",  # RIGHTNOW
        "ได้เลย ฉันจะเริ่มเดี๋ยวนี้",  # RIGHTNOW
        "พรุ่งนี้ตอนบ่ายสอง ฉันจะกำลังประชุมอยู่",  # LONGFUTURE
        "เวลานี้สัปดาห์หน้า เราจะเดินทางอยู่",  # LONGFUTURE
        "ฉันจะกำลังเรียนอยู่ตอนนั้น",  # LONGFUTURE
        "เขาจะกำลังนอนตอนเราไปถึง",  # LONGFUTURE
        "ภายในสิ้นปีนี้ ฉันจะเก็บเงินได้ 1 แสนบาท",  # PREDICT
        "เขาจะทำวิจัยเสร็จก่อนกำหนด",  # PREDICT
        "พรุ่งนี้ตอนเช้า เราจะถึงเชียงใหม่",  # PREDICT
        "ฉันจะเขียนรายงานเสร็จก่อน 5 โมงเย็น",  # PREDICT
        "ภายในปีหน้า ฉันจะทำงานที่นี่ครบ 10 ปี",  # WILLCONTINUEINFUTURE
        "ตอนนั้นเขาจะฝึกซ้อมมาแล้ว 6 เดือน",  # WILLCONTINUEINFUTURE
        "เมื่อถึงเวลานั้น เราจะคบกันมา 3 ปี",  # WILLCONTINUEINFUTURE
        "ฉันจะเรียนหลักสูตรนี้มาแล้ว 1 ปีเต็ม ในอีกสองเดือน",  # WILLCONTINUEINFUTURE
    ]

    random.seed(42)
    random.shuffle(test_sentences)

    all_results = []

    for sentence in test_sentences:
        result = system.full_pipeline(sentence, verbose=True)

        all_results.append({
            "thai_input": result["thai_input"],
            "english_translation": result["english_translation"],
            "coarse_label": result["tense_predictions"]["coarse"][0],
            "fine_label": result["tense_predictions"]["fine"][0],
            "confidence": result["tense_predictions"]["fine"][1],
            "translation_time_sec": result["timing"]["translation"],
            "explanation_time_sec": result["timing"]["explanation"],
            "total_time_sec": result["timing"]["total"],
            "explanation": result["explanation"]
        })

        print("\n" + "="*80 + "\n")

    # Save CSV to Google Drive path
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_path = f"/content/drive/MyDrive/Dissertation/Model v.5/results/hybrid4b_results_{timestamp}.csv"
    df = pd.DataFrame(all_results)
    df.to_csv(output_path, index=False)
    print(f"✅ Logged results saved to Google Drive:\n{output_path}")

In [ ]:
if __name__ == "__main__":
    import pandas as pd
    import random
    from datetime import datetime

    # Initialize system
    system = Hybrid4BSystem()

    # Load all models
    system.load_translator()
    system.load_classifier()
    system.load_explainer()

    # Extended test sentences
    test_sentences = [
        "ฉันกินข้าวเช้าทุกวัน",                            # HABIT
        "เราไปวิ่งตอนเช้าเป็นประจำ",                     #  HABIT
        "เขาตื่นสายเสมอ",                             # HABIT
        "คุณพ่อของผมตื่นนอนเวลา 8 โมงเช้า", # HABIT
        "น้ำเปล่าเดือดที่อุณหภูมิ 100 องศา",     # FACT
        "โลกหมุนรอบดวงอาทิตย์",     # FACT
        "ดวงอาทิตย์ขึ้นทิศตะวันออก",     # FACT
        "หนึ่งปีมีทั้งหมด 365 วัน",   # FACT
        "รถไฟออกตอน 9 โมง",         # SCHEDULEDFUTURE
    ]

    random.seed(42)
    random.shuffle(test_sentences)

    all_results = []

    for sentence in test_sentences:
        result = system.full_pipeline(sentence, verbose=True)

        all_results.append({
            "thai_input": result["thai_input"],
            "english_translation": result["english_translation"],
            "coarse_label": result["tense_predictions"]["coarse"][0],
            "fine_label": result["tense_predictions"]["fine"][0],
            "confidence": result["tense_predictions"]["fine"][1],
            "translation_time_sec": result["timing"]["translation"],
            "explanation_time_sec": result["timing"]["explanation"],
            "total_time_sec": result["timing"]["total"],
            "explanation": result["explanation"]
        })

        print("\n" + "="*80 + "\n")

    # Save CSV to Google Drive path
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_path = f"/content/drive/MyDrive/Dissertation/Model v.5/results/hybrid4b_results_{timestamp}.csv"
    df = pd.DataFrame(all_results)
    df.to_csv(output_path, index=False)
    print(f"✅ Logged results saved to Google Drive:\n{output_path}")

🔌 Loading GGUF translator...


llama_context: n_ctx_per_seq (4096) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_kv_cache_unified_iswa: using full-size SWA cache (ref: https://github.com/ggml-org/llama.cpp/pull/13194#issuecomment-2868343055)
llama_kv_cache_unified: LLAMA_SET_ROWS=0, using old ggml_cpy() method for backwards compatibility
llama_kv_cache_unified: LLAMA_SET_ROWS=0, using old ggml_cpy() method for backwards compatibility


✅ Translator loaded successfully
🔌 Loading BERT classifier...
✅ Classifier loaded successfully
🔌 Loading explainer model...


The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Explainer loaded successfully
📝 Input Thai: คุณพ่อของผมตื่นนอนเวลา 8 โมงเช้า

🔄 Step 1: Translation
   Result: My dad wakes up at 8 o'clock in the morning.
   Time: 1.82s

🏷️ Step 2: Tense Classification
   Top predictions:
   1. HABIT (Present Simple - กิจวัตร/นิสัย): 89.69%
   2. SCHEDULEDFUTURE (Present Simple - ตารางเวลา/แผนการที่กำหนดไว้): 9.24%
   3. FACT (Present Simple - ข้อเท็จจริง): 0.18%

💡 Step 3: Grammar Explanation
🔍 [Tense Prediction]
   Coarse: Present (99.89%)
   Fine  : HABIT (89.69%)


/usr/lib/python3.11/contextlib.py:105: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)
W0713 14:53:10.146000 229 torch/_inductor/utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode


✅ Explanation validation passed

📊 Total processing time: 99.40s

📖 TENSE DETAILS:

Tense Tag: HABIT
Tense: Present Simple
Thai Name: Present Simple - กิจวัตร/นิสัย
Usage: ใช้เมื่อพูดถึงกิจวัตรหรือพฤติกรรมที่ทำเป็นประจำ
Structure: Subject + V1 (ถ้าประธานเอกพจน์เติม s/es)
Keywords: always, usually, often, sometimes, every day
Example: I drink coffee every morning.

📖 EXPLANATION:
**1. วิเคราะห์ Tense ที่ใช้:**

*   **Tense ที่ใช้:** ประโยคนี้ใช้ **Present Simple** (ปัจจุบันบุคลิก)
*   **การใช้งานในบริบทนี้:** Present Simple ถูกใช้เพื่อบรรยายกิจวัตรประจำวันของบุคคล ซึ่งในกรณีนี้คือการตื่นนอนของ "คุณพ่อ" ของผู้พูด การตื่นนอนเวลา 8 โมงเช้าเป็นสิ่งที่เกิดขึ้นเป็นประจำ
*   **โครงสร้างไวยากรณ์:**  โครงสร้างหลักคือ “Subject + Verb” โดยในประโยคนี้ Subject คือ “My dad” (คุณพ่อของผม) และ Verb คือ “wakes up” (ตื่นนอน) เนื่องจากประธานเป็นเอกพจน์ (dad) จึงไม่จำเป็นต้องเติม s/es

**2. คำศัพท์ที่น่าสนใจ:**

*   **“in the morning”**:  วลีนี้มีความสำคัญในการระบุช่วงเวลาที่แน่นอนของการตื่นนอน  การใช้ “in

/usr/lib/python3.11/contextlib.py:105: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)


✅ Explanation validation passed

📊 Total processing time: 17.62s

📖 TENSE DETAILS:

Tense Tag: FACT
Tense: Present Simple
Thai Name: Present Simple - ข้อเท็จจริง
Usage: ใช้กับข้อเท็จจริงที่เป็นสัจธรรมหรือเป็นความรู้ทางวิทยาศาสตร์
Structure: Subject + V1
Keywords: ข้อเท็จจริงทั่วไป, สัจธรรม
Example: The sun rises in the east.

📖 EXPLANATION:
**การวิเคราะห์ Tense ในประโยค “ดวงอาทิตย์ขึ้นทิศตะวันออก”**

1.  **วิเคราะห์ Tense ที่ใช้:**

    *   Tense ที่ใช้ในประโยค "The sun rises in the east." คือ **Present Simple**
    *   **การใช้งาน:** Present Simple ใช้เพื่อแสดงข้อเท็จจริงที่เป็นสัจธรรม หรือความรู้ทางวิทยาศาสตร์ ซึ่งเป็นลักษณะของประโยคนี้ ดวงอาทิตย์ขึ้นทิศตะวันออกเป็นปรากฏการณ์ธรรมชาติที่มีเกิดขึ้นทุกวัน เป็นความจริงที่ไม่เปลี่ยนแปลง
    *   **โครงสร้างไวยากรณ์:** ประโยคนี้มีโครงสร้าง Subject + Verb (V1) ได้แก่ “The sun” (Subject) และ “rises” (Verb)

2.  **คำศัพท์ที่น่าสนใจ:**

    *   คำที่น่าสนใจคือ “rises” (กริยาช่องที่สาม) การใช้ Present Simple กับกริยาช่องที่สาม (แบบ third form) แ